# Reed-Solomon Error Correction
This noteboook shows how to use the Reed-Solomon error corrector.

Start by importing necessary modules:

In [1]:
from ecc.reed_solomon import RS
from ecc.message import Message

### Message ###

If you read the [hamming.ipynb](hamming.ipynb) notebook you saw that the Message class allows conversion from strings to bits and viceversa. However, for Reed-Solomon, instead of encoding/decoding bits, we will be using bytes by means of the `bits2int` method, which will interpret 8 consecutive bits (a byte) as an unsigned integer. 

Let's check it out.

In [2]:
msg = Message()
message = 'áéíóú'
bits = msg.to_bits(message)
_ints = msg.bits2int(bits)
print(f"Message: {message}")
print(f"bits   : {bits}")
print(f"Bytes  : {_ints}")
print(f"\nMessage: {msg.from_bits(msg.int2bits(_ints))}")

Recall the method `negate` allows to negate specified bits of a bit array.

In [3]:
altered_bits = msg.negate(bits, 3, 4, 7, 23, -9, -3, -2, -1)
altered_ints = msg.bits2int(altered_bits)
print(f"bits   : {altered_bits}")
print(f"Bytes  : {altered_ints}")
print(f"\nMessage: {msg.from_bits(msg.int2bits(altered_ints))}")

### Reed-Solomon codes ###

This notebook is not intended to teach how Reed-Solomon codes work. They are a bit complicated and some additional mathematics knowledge is required. 

Reed–Solomon codes operate on a block of data treated as a set of finite-field elements called symbols.  [<sup>1</sup>](#fn1 "https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction"). 
In this implementation of Reed-Solomon codes, each symbol will be an element of a Galois Field (2⁸). The elements of this finite field can only take integer values from 0 to 255. The symbols used will be the bytes (they are actually python ints but are built from every 8 bits in a bit array) given by the `Message.bits2int` method mentioned before.

 
The class `GF_256` provides us with the arithmetic needed for Reed-Solomon coding. To keep things simple, although not the most efficient approach, a list of integers is considered to be a polynomial, the first elements being the higher order terms of the polynomial. For example, 8x⁶ + 3x⁴ + 240x² + x + 156 is represented as [8,0,3,0,240,1,156]. 

Polynomial addition, subtraction, multiplication, division, modular division and exponentiation as well as other necessary operations are implemented here. The irreducible polynomial that I have used for the Galois Field (2⁸) is x⁸+x⁴+x³+x+1. All the pydantic `@validate_arguments` decorators in the `GF_256` class are commented to speed up the Reed-Solomon coding/decoding algorithms, but can be uncommented if desired. Let's see how it works.
 

<span id="fn1">1: https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction</span>


In [4]:
from ecc.reed_solomon import GF_256

f = GF_256(220)
a = [75, 0, 127, 240, 3]  # 75x⁴ + 127x² + 240x +3
b = [4, 92, 13]  # 4x² + 92x + 13

added = f.gf_256_pol_sum(a, b)
subtracted = f.gf_256_pol_subtract(a, b)
multiplied = f.gf_256_pol_multiply(a, b)
divided = f.gf_256_pol_divide(a, b)

print(f"Polynomials: {a} and {b}")
print(f"Added      : {added}")
print(f"Subtracted : {subtracted}")
print(f"Multiplied : {multiplied}")
print(f"Divided    : {divided}")

I found [this](http://www.ee.unb.ca/cgi-bin/tervo/calc2.pl) online calculator from the University of New Brunswick that helped me checking my results while defining these operations. Try it out (don't forget to set x⁸+x⁴+x³+x+1 as the irreducible polynomial if you want to get the same results as me).

Having defined arithmetic operations in Galois Field (2⁸) a Reed-Solomon error corrector can be implemented. The `RS` class takes care of this, just instantiate it by passing `k` (the max data length per block) as arguement. No need to worry about polynomials or weird mathematics thanks to the `encode` and `decode` methods. Simply pass a list of integers that represent a message. 

Let's try it out with the same message from before. Recall that Reed-Solomon adds *n - k + 1* redundacy coefficients to a block of data, where *n* is the block length (always 255 in `RS`) and *k* in the following example is 223. To guaranee that the same bits are altered, the -9,-3,-2,-1 bits of the message to be negated are now *(n - k + 1) x 8* bits earlier, in this example 264, so their positions in the encoded message are actually -273, -267, -266, -265. 

In [5]:
message = 'áéíóú'
_ints = msg.bits2int(msg.to_bits(message))

rs = RS(223)
enc_bits = msg.int2bits(rs.encode(_ints))
altered_bits = msg.negate(enc_bits, 3, 4, 7, 23, -273, -267, -266, -265)
dec_ints = rs.decode(msg.bits2int(altered_bits))
dec_message = msg.from_bits(msg.int2bits(dec_ints))

print(f"Message         : {message}")
print(f"Bytes           : {_ints}")
print(f"Sent bytes      : {rs.encode(_ints)}")
print(f"Received bytes  : {msg.bits2int(altered_bits)}")
print(f"\nDecoded message : {dec_message}")

Message         : áéíóú
Bytes           : [195, 161, 195, 169, 195, 173, 195, 179, 195, 186]
Sent bytes      : [195, 161, 195, 169, 195, 173, 195, 179, 195, 186, 10, 178, 129, 209, 206, 42, 255, 44, 203, 26, 165, 213, 11, 234, 24, 250, 210, 154, 159, 100, 106, 21, 144, 233, 155, 126, 41, 112, 129, 17, 107, 130, 195]
Received bytes  : [218, 161, 194, 169, 195, 173, 195, 179, 194, 189, 10, 178, 129, 209, 206, 42, 255, 44, 203, 26, 165, 213, 11, 234, 24, 250, 210, 154, 159, 100, 106, 21, 144, 233, 155, 126, 41, 112, 129, 17, 107, 130, 195]

Decoded message : áéíóú


Let's try a longer message.

In [6]:
message = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Tempus iaculis urna id volutpat. Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit.'
enc_bits = msg.int2bits(rs.encode(msg.bits2int(msg.to_bits(message))))

Now alter some bits. Since bit errors tend to come in burst in practice, a generator will be used to simulate burst errors of 24 bits.

In [7]:
from itertools import chain

negate = range(0, len(enc_bits), 24)
negate = chain(range(negate[0], negate[1]), range(negate[8], negate[9]), range(negate[18], negate[19]), range(negate[23], negate[24]), range(negate[-2], negate[-1]))
negate = [x for x in negate]
print("Bits to be altered:")
print(*negate)
print(f"\nTotal altered bits: {len(negate)}")

Bits to be altered:
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 2280 2281 2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295 2296 2297 2298 2299 2300 2301 2302 2303

Total altered bits: 120


As can be seen, plenty of bits will be altered. 

Anyways, Reed-Solomon is powerful enough to correct them.

In [8]:
received_bits = msg.negate(enc_bits, *negate)
dec_message = msg.from_bits(msg.int2bits(rs.decode(msg.bits2int(received_bits))))

print(f'Hash of original message: {hash(message)}')
print(f'Hash of decoded message:  {hash(dec_message)}')

Hash of original message: -1558771434150372974
Hash of decoded message:  -1558771434150372974
